<a href="https://colab.research.google.com/github/koyonashi-llc/gcp_text-to-speech_colab/blob/main/GCP_Text_to_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Cloud Text-to-Speech による音声データ作成

入力された日本語または英語を mp3 形式の音声データで出力します。

事前に Google Cloud Platform Console で API の有効化、サービスアカウントの作成および認証キーの作成が必要です。  
詳しくは、Cloud Text-to-Speech のドキュメント「[始める前に](https://cloud.google.com/text-to-speech/docs/before-you-begin)」をご確認ください。

作成した認証キー（jsonファイル）は、Googleドライブの任意の場所へ保存してください。

## 必要パッケージをインストールする。

※パッケージをインストールした時点で一度ランタイムを再起動（Ctrl+M .）してください。

In [ ]:
pip install --upgrade --no-cache-dir google-cloud-texttospeech

## Googleドライブを利用可能にする。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 認証キーの保存場所を指定する。

'/content/drive/My Drive/xxxxx/xxxxx.json' の形式で、認証キーを保存したGoogleドライブ上のパスを指定してください。

（例）Googleドライブのマイドライブ直下に 'key.json' というファイル名で保存している場合  
　'/content/drive/My Drive/key.json'

In [ ]:
#@title **認証キーの保存場所**
credential_path = "/content/drive/My Drive/Colab Notebooks/text-to-speech/xxxxxxxx.json" #@param {type:"string"}

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

## 音声データの作成処理を定義する。

コードに関する詳細は以下のドキュメントを参照してください。

- [すべての Text-to-Speech のコードサンプル](https://cloud.google.com/text-to-speech/docs/samples)

In [ ]:
def synthesize_text(text,name,speakingRate,pitch,effectsProfileId,audio_encoding):
    """Synthesizes speech from the input string of text."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    input_text = texttospeech.SynthesisInput(text=text)

    # Note: the voice can also be specified by name.
    # Names of voices can be retrieved with client.list_voices().
    voice = texttospeech.VoiceSelectionParams(
        language_code=name[:5],
        name=name,
        ssml_gender=texttospeech.SsmlVoiceGender.SSML_VOICE_GENDER_UNSPECIFIED
    )

    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3 if audio_encoding=='MP3' else texttospeech.AudioEncoding.OGG_OPUS,
        speaking_rate=speakingRate,
        pitch=pitch,
        effects_profile_id=[effectsProfileId]
    )

    response = client.synthesize_speech(
        request={"input": input_text, "voice": voice, "audio_config": audio_config}
    )

    # The response's audio_content is binary.
    with open("output.mp3" if audio_encoding=='MP3' else "output.ogg", "wb") as out:
        out.write(response.audio_content)
        print('Audio content written to file ' + '"output.mp3"' if audio_encoding=='MP3' else '"output.ogg"')


## 音声を作成する。

設定情報へ必要事項を入力してから実行してください。  
処理が完了すると、'content' ディレクトリ直下に 'output.mp3' ファイルが生成されますのでダウンロードしてご利用ください。

設定情報の詳細は以下のドキュメントを参照してください。

- [Method: text.synthesize](https://cloud.google.com/text-to-speech/docs/reference/rest/v1/text/synthesize)
- [サポートされている音声と言語](https://cloud.google.com/text-to-speech/docs/voices)
- [生成された音声へのデバイス プロファイルの使用](https://cloud.google.com/text-to-speech/docs/audio-profiles)


In [ ]:
#@title **作成したい音声の設定**
# SynthesisInput
text = "\u73FE\u5728\u3001\u97F3\u58F0\u3092\u6271\u3046\u4ED5\u4E8B\u3092\u3057\u3066\u3044\u308B\u4EBA\u306B\u304A\u52E7\u3081\u3057\u307E\u3059\u3002\u81EA\u7136\u8A00\u8A9E\u51E6\u7406\u306B\u643A\u308F\u3063\u3066\u3044\u308B\u4EBA\u306B\u3082\u3002\u307E\u305F\u3001\u8B1B\u5E2B\u306E\u65B9\u3067\u3042\u308C\u3070\u3001\u8B1B\u7FA9\u3092\u52D5\u753B\u306B\u3059\u308B\u306E\u306B\u6700\u9069\u306A\u65B9\u6CD5\u3067\u3057\u3087\u3046\u3002\u8B1B\u7FA9\u3092\u306F\u3058\u3081\u3001\u3042\u3089\u3086\u308B\u30C6\u30AD\u30B9\u30C8\u3092\u7FFB\u8A33\u3067\u304D\u308B\u3068\u3044\u3046\u610F\u5473\u3067\u3082\u3001\u3053\u308C\u306F\u3059\u3054\u3044\u3067\u3059\u3088" #@param {type:"string"}
# VoiceSelectionParams
name = "ja-JP-Wavenet-C" #@param ["en-US-Wavenet-B", "en-US-Wavenet-F", "ja-JP-Wavenet-C", "ja-JP-Wavenet-B", "", "", "", "", ""]
# AudioConfig
speakingRate = 1 #@param {type:"slider", min:0.25, max:4.0, step:0.1}
pitch = 0 #@param {type:"slider", min:-20, max:20, step:0.1}
effectsProfileId = "headphone-class-device" #@param ["wearable-class-device", "handset-class-device", "headphone-class-device", "small-bluetooth-speaker-class-device", "medium-bluetooth-speaker-class-device", "large-home-entertainment-class-device", "large-automotive-class-device", "telephony-class-application"]
# AudioEncoding
audio_encoding = "OGG_OPUS" #@param ["MP3", "OGG_OPUS"]


synthesize_text(text,name,speakingRate,pitch,effectsProfileId,audio_encoding)